In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel,BertTokenizerFast
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
path = config.path + 'MIND/MINDdemo_train/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

[2021-08-29 09:47:17,329] INFO (utils.MIND) encoding user behaviors of ../../../Data/MIND/MINDdemo_train/behaviors.tsv...
[2021-08-29 09:47:17,481] INFO (utils.MIND) loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-08-29 09:47:18,123] INFO (utils.MIND) reducing news of ../../../Data/MIND/MINDdemo_train/news.tsv...
[2021-08-29 09:47:18,124] INFO (utils.utils) unmasking at least k...
[2021-08-29 09:47:18,304] INFO (utils.utils) deduplicating...


In [ ]:
a[0]['cdd_subword_index'].shape

In [3]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False)
records1 = list(loader1)

In [7]:
records1[0]['his_attn_mask']

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [18]:
sub = a.subwords[1:3][:,10:20]
index = sub[:,:,0] * 10 + sub[:,:,1]

dest = torch.zeros((2, 10 * 10))
index

tensor([[110, 121, 132, 143, 154, 165, 166, 167, 178, 189],
        [ 90, 101, 112, 123, 134, 145, 156, 167, 178, 189]])

In [28]:
dest[torch.arange(2), ]

tensor([1., 0.])

In [ ]:
dic = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_dev/10/behaviors.pkl', 'rb'))
dic['imprs']

In [7]:
t = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [37]:
res = t(["[CLS] Newark Liberty Airport's Terminal One a $2.7 billion 'transformative' project","shit bro"], add_special_tokens=False, padding=True, max_length=20)
res

{'input_ids': [[101, 12948, 7044, 3199, 1005, 1055, 5536, 2028, 1037, 1002, 1016, 1012, 1021, 4551, 1005, 10938, 8082, 1005, 2622], [4485, 22953, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [21]:
re.sub('##','',' '.join(tokens))

"[CLS] newark liberty airport ' s terminal one a $ 2 . 7 billion ' transform ative ' project ?"

In [30]:
words = []
for tok in tokens:
    if tok.startswith('##'):
        words[-1] += tok[2:]
    else:
        words.append(tok)

In [48]:
' '.join(np.array(['I','you','she'],dtype=object))

'I you she'

In [42]:
res.token_to_word()

TypeError: token_to_word() missing 1 required positional argument: 'batch_or_token_index'